In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle
import datetime as dt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

from IdealDataInterface import IdealDataInterface
from IdealMetadataInterface import IdealMetadataInterface
from dateutil.relativedelta import relativedelta
from datetime import timedelta

### Specify the folder where your sensor data is

In [4]:
df = pickle.load(open("D:/Documents/MSc Project/data/reduced_data.p", 'rb'))

In [6]:
result = []
for i in df:
    result.append(len(df[i]))

In [8]:
np.mean(result)

11445.833333333334

In [9]:
df

{59:                      electric-combined
 time                                  
 2016-10-06 09:00:00           0.235403
 2016-10-06 10:00:00           0.241304
 2016-10-06 11:00:00           0.592701
 2016-10-06 12:00:00           0.226969
 2016-10-06 13:00:00           0.143887
 ...                                ...
 2018-01-28 03:00:00           0.128288
 2018-01-28 04:00:00           0.125962
 2018-01-28 05:00:00           0.123077
 2018-01-28 06:00:00           0.144816
 2018-01-28 07:00:00           0.146810
 
 [11495 rows x 1 columns],
 61:                      electric-combined
 time                                  
 2016-10-06 12:00:00           0.194816
 2016-10-06 13:00:00           0.216332
 2016-10-06 14:00:00           0.307932
 2016-10-06 15:00:00           0.087980
 2016-10-06 16:00:00           0.079463
 ...                                ...
 2017-10-19 02:00:00           0.086417
 2017-10-19 03:00:00                NaN
 2017-10-19 04:00:00           0.219398
 20

In [4]:
for i in df:
    df[i]['day'] = df[i].index.dayofweek

In [5]:
def increment_month(date):
    if (date + timedelta(1)).month == date.month:
        return date + relativedelta(months=1)
    return (date + relativedelta(months=2)).replace(day=1) - timedelta(1)

In [6]:
def get_cut(data, start, end):
    'Function to return a ful cut of data for that month depending on year'
    
    cut = data[start : end]
    
    size = cut['day'].size
    
    lastdate = end
    
    if not cut.empty:
        lastdate = cut.iloc[size - 1].name.to_pydatetime()
    
    if cut.empty or lastdate < end:
        
        start -= relativedelta(years=1)
        end -= relativedelta(years=1)

        cut = data[start : end]
        
    return cut

In [7]:
def aggregate_days(data):
    'Function to take the average data for each hour of all weekdays and weekends'
    
    'Initiate first and last days of the month as datetime'
    startdate = datetime.datetime(2017, 1, 31)
    enddate = datetime.datetime(2017, 1, 31)
    
    'Create empty dictionary to contain result'
    result = {}
    
    'If the first day in the dataset is after 1/1/2017, make start date 1/1/2018'
    if data.index.min() > startdate:
        
        startdate = datetime.datetime(2018, 1, 1)
        enddate = datetime.datetime(2018, 1, 31)
    else:

        startdate = datetime.datetime(2017, 1, 1)

    'Iterate through 12 months and aggregate weekday and weekend data'
    for i in range(12):
        
        'Get one month cut of the data using start and end dates of month'
        df_cut = get_cut(data, startdate, enddate)            
    
        'Seperate weekday and weeekend data into different DataFrames'
        weekday = df_cut.loc[df_cut['day'] < 5]
        weekend = df_cut.loc[df_cut['day'] > 4]
        
        'Remove the day identifier column'
        weekday.drop('day', axis=1, inplace=True)
        weekend.drop('day', axis=1, inplace=True)
        
        'Group all data by hour and take the mean of each hour'        
        weekday = weekday.groupby([weekday.index.hour]).mean()
        weekend = weekend.groupby([weekend.index.hour]).mean()
        
        result[i] = {0: weekday, 1: weekend}
        
        'Move to next month'
        startdate = increment_month(startdate)
        enddate = increment_month(enddate)
        
    return result

In [8]:
def aggregate_house_data(data):
    'Function to create final data for model run'
    
    result = {}
    
    for i in data:
        result[i] = aggregate_days(data[i])
        
    return result

In [9]:
def empty_df_check(data):
    'Function to check for any empty dataframes in data'
    empty = []
    for i in data.keys():
        for x in range(12):
            for n in range(2):
                if data[i][x][n].empty:
                    empty.append({"house": i, "month": x, "day": n})
                    
    return empty

In [10]:
def get_means(data):
    'Function to get the mean values for each out hour, each aggreagted weekday and weekend for each month'
    
    means = {}

    for i in range(12):
        means[i] = {0:[], 1:[]}
        for x in data.keys():
            for n in range(2):
                means[i][n].append(data[x][i][n])
                
    for i in means.keys():
        for n in range(2):
            means[i][n] = pd.concat(means[i][n], axis=1)
            means[i][n]['mean'] = means[i][n].mean(axis=1)
            means[i][n] = means[i][n]['mean']
            
    return means

In [11]:
def null_check(data):
    'Function to check for any null values in the aggregated data'
    
    null_check = {}
    for i in data.keys():
        nulls = 0
        for x in range(12):
            for n in range(2):
                nulls += data[i][x][n].isnull().sum()

        null_check[i] = nulls
    
    result = pd.DataFrame(null_check).swapaxes("index", "columns")
    
    return result

In [12]:
def fill_null(data, means):
    'Function to fill missing values with the averages of other houses at that time'
    
    for i in data.keys():
        for n in range(12):
            for x in range(2):
                data[i][n][x][data[i][n][x].isna()] = pd.DataFrame(means[n][x]).values
                
    return data

In [13]:
def prep_data(data):
    'Function to prepare the data for simulation'
    
    'Add column to indicate the day of the week'
    for i in data:
        data[i]['day'] = data[i].index.dayofweek
    
    agg_data = aggregate_house_data(data)
    
    mean_data = get_means(agg_data)
    
    if len(empty_df_check(agg_data)) > 0:
        return "Empty dataframes present"
    
    null_count = null_check(agg_data).sum().iloc[0]
    
    if null_count > 0:
        agg_data = fill_null(agg_data, mean_data)
        
    return agg_data

In [14]:
data = prep_data(df)

C:\Users\Ellio\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
pickle.dump(data, open('aggregated_data.p', 'wb'))